In [1]:
import pandas as pd
from joblib import dump
import uvicorn
from fastapi import FastAPI

import numpy as np
import pickle
import pandas as pd


import sys
sys.path.insert(0, '../fraud-detector/')
sys.path.insert(0, '../data')
sys.path.insert(0, '../')

import conf.scoring_conf as conf
from utils import load_data_local
from feateng import FeatPipeline
import joblib

###############################
# 0. Variable Assignment
###############################

inpath = conf.toscore_inpath # input path of data to be scored
intype = conf.toscore_intype # input path type of data to be scored
model_chosen = conf.model_chosen

###############################
# 1. Data read
###############################
df = load_data_local(inpath, intype)

###############################
# 2. Feature engineering pipeline
###############################
df2 = FeatPipeline.fit_transform(df)

###############################
# 3. Scoring
###############################
model = joblib.load(model_chosen+ ".pkl")

# Convert data to json
target_feature = ["isFraud"]
id_features = ['nameDest', 'nameOrig']
all_features = df.columns.difference(target_feature).difference(id_features)
data = pd.DataFrame.to_json(df2[all_features])

from pydantic import BaseModel

# TODO: make configurable
class Transaction(BaseModel):
    amount: float
    isDestBalanceNewZero: int
    isDestBalanceOldZero: int
    isDestMerchant: int
    isOrigBalanceNewZero : int
    isOrigBalanceOldZero : int
    newbalanceDest : float
    newbalanceOrig : float
    oldbalanceDest : float
    oldbalanceOrg : float
    step: int
    stepDay: int
    stepHour: int
    stepWeekDay: int
    type: str


In [2]:
df2[all_features].head()

,amount,isDestBalanceNewZero,isDestBalanceOldZero,isDestMerchant,isFlaggedFraud,isOrigBalanceNewZero,isOrigBalanceOldZero,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,stepDay,stepHour,stepWeekDay,type
0,309128.55,0,0,0,0,1,1,743261.26,0.0,434132.72,0.00,321,14.0,9.0,0.0,CASH_OUT
1,1035588.40,0,1,0,0,1,0,1035588.40,0.0,0.00,25691.00,154,7.0,10.0,0.0,TRANSFER
2,320499.65,0,0,0,0,0,0,85338.15,8646714.8,405837.80,8326215.15,348,15.0,12.0,1.0,CASH_IN
3,272780.92,0,0,0,0,1,1,928395.54,0.0,655614.62,0.00,262,11.0,22.0,4.0,TRANSFER
4,11310.62,0,0,0,0,1,1,1068365.73,0.0,1057055.11,0.00,227,10.0,11.0,3.0,CASH_OUT


In [6]:
{
  "amount": 309128.55,
  "isDestBalanceNewZero": 0,
  "isDestBalanceOldZero": 0,
  "isDestMerchant": 0,
  "isOrigBalanceNewZero": 1,
  "isOrigBalanceOldZero": 1,
  "newbalanceDest": 743261.26,
  "newbalanceOrig": 0.0,
  "oldbalanceDest": 434132.72,
  "oldbalanceOrg": 0.00,
  "step": 321,
  "stepDay": 14.0,
  "stepHour": 9.0,
  "stepWeekDay": 0.0,
  "type": "CASH_OUT"
}


'{'

In [11]:
model.predict(df2[:3][all_features])

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


array([0, 0, 0])

In [12]:
pd.DataFrame.to_json(df2[:3][all_features])

'{"amount":{"0":309128.55,"1":1035588.4,"2":320499.65},"isDestBalanceNewZero":{"0":0,"1":0,"2":0},"isDestBalanceOldZero":{"0":0,"1":1,"2":0},"isDestMerchant":{"0":0,"1":0,"2":0},"isFlaggedFraud":{"0":0,"1":0,"2":0},"isOrigBalanceNewZero":{"0":1,"1":1,"2":0},"isOrigBalanceOldZero":{"0":1,"1":0,"2":0},"newbalanceDest":{"0":743261.26,"1":1035588.4,"2":85338.15},"newbalanceOrig":{"0":0.0,"1":0.0,"2":8646714.8000000007},"oldbalanceDest":{"0":434132.72,"1":0.0,"2":405837.8},"oldbalanceOrg":{"0":0.0,"1":25691.0,"2":8326215.1500000004},"step":{"0":321,"1":154,"2":348},"stepDay":{"0":14.0,"1":7.0,"2":15.0},"stepHour":{"0":9.0,"1":10.0,"2":12.0},"stepWeekDay":{"0":0.0,"1":0.0,"2":1.0},"type":{"0":"CASH_OUT","1":"TRANSFER","2":"CASH_IN"}}'